In [2]:
import numpy as np
import pandas as pd
import sqlite3

In [3]:
data = pd.read_csv('./age-education.csv')

In [17]:
conn = sqlite3.connect("population.db")
c = conn.cursor()

In [18]:
def drop_tables(c):
    
    tables = ['AREA','AGE_LITERACY','LANGUAGE_AGE','EDUCATION','LANG_EDUCATION']
    try:
        for t in tables:
            sql = '''DROP TABLE '''+ t
            c.execute(sql)
    except sqlite3.IntegrityError as e:
        print('sqlite error: ', e.args[0]) # column name is not unique
    conn.commit()
    print('done')

In [19]:
drop_tables(c)

done


In [8]:
def create_table(c,query,name):
    try:
        c.execute(query)
    except sqlite3.IntegrityError as e:
        print('sqlite error: ', e.args[0]) # column name is not unique
    conn.commit()
    print(name,'done')

In [9]:
def insert_many(c,schema,values,table):
    try:
        sql = '''INSERT INTO '''+table + schema[0] + ''' VALUES '''+ schema[1]
        c.executemany(sql, values)
    except sqlite3.IntegrityError as e:
        print('sqlite error: ', e.args[0]) # column name is not unique
    conn.commit()
    print('Insertion done')

In [10]:
def avg_age(tup):
#     print(tup)
    if tup=='Total' or tup == 'Age not stated' or tup == 'All ages':
        return -1
    elif '+' in tup:
        return int(tup[:-1])+5
    elif '-' in tup:
        a_b = tup.split("-")
#         print(a_b)
        return (int(a_b[0]) + int(a_b[1]))//2
    else:
        return int(tup)


## AREA TABLE

In [11]:
table_1 = '''CREATE TABLE AREA(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
STATE_CODE INT NOT NULL,
DISTT_CODE INT NOT NULL,
NAME TEXT NOT NULL);'''
create_table(c,table_1,'AREA')

done


In [12]:
area_db = data.iloc[6:,1:4].drop_duplicates().values

In [8]:
# area_db

In [14]:
schema =['''(STATE_CODE,DISTT_CODE,NAME)''','''(?,?,?)''']
insert_many(c,schema,area_db,'AREA')

done


## AGE_LITERACY TABLE

In [18]:
table_3 = '''CREATE TABLE AGE_LITERACY(
ID INTEGER PRIMARY KEY AUTOINCREMENT,
STATE_CODE INTEGER NOT NULL,
GEOGRAPHIC_AREA TEXT NOT NULL,
AGE_GROUP TEXT,
TOTAL_PERSONS INTEGER,TOTAL_MALE INTEGER,TOTAL_FEMALE INTEGER,
ILLITERATE_PERSONS INTEGER,ILLITERATE_MALE INTEGER,ILLITERATE_FEMALE INTEGER,LITERATE_PERSONS INTEGER,LITERATE_MALE INTEGER,LITERATE_FEMALE INTEGER,
AVG_AGE INTEGER,
FOREIGN KEY (STATE_CODE) REFERENCES AREA(STATE_CODE)
);'''
create_table(c,table_3,'AGE_LITERACY')

done


In [19]:
# data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,C-8 EDUCATIONAL LEVEL BY AGE AND SEX FOR POPULATION AGE 7 AND ABOVE - 2011,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,Table Name,State code,District code,Area Name,Total/Rural/Urban,Age-group,Total,NaN,NaN,Illiterate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Technical diploma or certificate,NaN,NaN,Graduate & above,NaN,NaN,Unclassified,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,not equal to degree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Persons,Males,Females,Persons,...,Females,Persons,Males,Females,Persons,Males,Females,Persons,Males,Females
4,NaN,NaN,NaN,NaN,NaN,1,2,3,4,5,...,31,32,33,34,35,36,37,38,39,40


In [20]:
age_lit = data.iloc[6:,[1,4,5,6,7,8,9,10,11,12,13,14]]
age_lit['avg_age']=age_lit.iloc[:,2].map(avg_age)

In [21]:
age_lit = age_lit.fillna(0)

In [23]:
schema_3 =['''(STATE_CODE,GEOGRAPHIC_AREA,AGE_GROUP,TOTAL_PERSONS,TOTAL_MALE,TOTAL_FEMALE,ILLITERATE_PERSONS,ILLITERATE_MALE,ILLITERATE_FEMALE,LITERATE_PERSONS,LITERATE_MALE,LITERATE_FEMALE,AVG_AGE)''','''(?,?,?,?,?,?,?,?,?,?,?,?,?)''']
insert_many(c,schema_3,age_lit.values,'''AGE_LITERACY''')

done


## Education Table

In [43]:
table_4 = '''CREATE TABLE EDUCATION(
STATE_CODE INTEGER NOT NULL,
GEOGRAPHIC_AREA TEXT NOT NULL,
AGE_GROUP TEXT,
LITERATE_WITHOUT_EDUCATION_TOTAL INTEGER,LITERATE_WITHOUT_EDUCATION_MALE INTEGER,LITERATE_WITHOUT_EDUCATION_FEMALE INTEGER,
BELOW_PRIMARY_TOTAL INTEGER,BELOW_PRIMARY_MALE INTEGER,BELOW_PRIMARY_FEMALE INTEGER,
PRIMARY_TOTAL INTEGER,PRIMARY_MALE INTEGER,PRIMARY_FEMALE INTEGER,
MIDDLE_TOTAL INTEGER,MIDDLE_MALE INTEGER,MIDDLE_FEMALE INTEGER,
SECONDARY_TOTAL INTEGER,SECONDARY_MALE INTEGER,SECONDARY_FEMALEINTEGER,
HIGHER_SECONDARY_TOTAL INTEGER,HIGHER_SECONDARY_MALE INTEGER,HIGHER_SECONDARY_FEMALE INTEGER,
NON_TECHNICIAN_TOTAL INTEGER,NON_TECHNICIAN_MALE INTEGER,NON_TECHNICIAN_FEMALE INTEGER,
TECHNICIAN_TOTAL INTEGER,TECHNICIAN_MALE INTEGER,TECHNICIAN_FEMALE INTEGER,
GRADUATE_TOTAL INTEGER,GRADUATE_MALE INTEGER,GRADUATE_FEMALE INTEGER,
UNCLASSIFIED_TOTAL INTEGER,UNCLASSIFIED_MALE INTEGER,UNCLASSIFIED_FEMALE INTEGER,
FOREIGN KEY (STATE_CODE) REFERENCES AREA(STATE_CODE)
);'''
create_table(c,table_4)

done


In [44]:
ids = [1,4,5]+[i for i in range(15,len(data.columns))]
edu = data.iloc[6:,ids]

In [45]:
edu = edu.fillna(0)

In [47]:
schema_3 =['''''','''(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''']
insert_many(c,schema_3,edu.values,'''EDUCATION''')

done


## Language_Age Table

In [48]:
table_5 = '''CREATE TABLE LANGUAGE_AGE(
STATE_CODE INTEGER NOT NULL,
GEOGRAPHIC_AREA TEXT NOT NULL,
AGE_GROUP TEXT,
SECOND_LANGUAGE_TOTAL INTEGER,SECOND_LANGUAGE_MALE INTEGER,SECOND_LANGUAGE_FEMALE INTEGER,
THIRD_LANGUAGE_TOTAL INTEGER,THIRD_LANGUAGE_MALE INTEGER,THIRD_LANGUAGE_FEMALE INTEGER,AVG_AGE INTEGER,
FOREIGN KEY (STATE_CODE) REFERENCES AREA(STATE_CODE),
PRIMARY KEY (STATE_CODE, GEOGRAPHIC_AREA, AGE_GROUP)
);'''
create_table(c,table_5,'LANGUAGE_AGE')

done


In [49]:
data_2 = pd.read_csv('./multilingual-age.csv')
data_2.fillna(0)

,"C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State code,District code,Area Name,Total/Rural/Urban,Age-group,Number speaking second language,0,0,Number speaking third language,0,0
1,0,0,0,0,0,Persons,Males,Females,Persons,Males,Females
2,1,2,3,4,5,6,7,8,9,10,11
3,0,0,0,0,0,0,0,0,0,0,0
4,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
5,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957
6,00,00,INDIA,Total,10-14,34488492,18133423,16355069,7254335,3831131,3423204
7,00,00,INDIA,Total,15-19,42424599,22750908,19673691,12626717,6792766,5833951
8,00,00,INDIA,Total,20-24,41344406,22386694,18957712,12834334,7067614,5766720
9,00,00,INDIA,Total,25-29,35924045,19695033,16229012,10857171,6144045,4713126


In [50]:
ids = data_2.iloc[4:,[0,3,4,5,6,7,8,9,10]]

In [51]:
def avg_age(tup):
#     print(tup)
    if tup=='Total' or tup == 'Age not stated':
        return -1
    elif '+' in tup:
        return int(tup[:-1])+5
    else:
        a_b = tup.split("-")
#         print(a_b)
        return (int(a_b[0]) + int(a_b[1]))//2
ids['avg_age']=ids.iloc[:,2].map(avg_age)

In [53]:
schema_4 =['''''','''(?,?,?,?,?,?,?,?,?,?)''']
insert_many(c,schema_4,ids.values,'''LANGUAGE_AGE''')

done


## Language_Education Table

In [54]:
table_6='''CREATE TABLE LANG_EDUCATION(
STATE_CODE INTEGER NOT NULL,
GEOGRAPHIC_AREA TEXT NOT NULL,
EDUCATION_LEVEL TEXT,
SECOND_LANGUAGE_TOTAL INTEGER,SECOND_LANGUAGE_MALE INTEGER,SECOND_LANGUAGE_FEMALE INTEGER,
THIRD_LANGUAGE_TOTAL INTEGER,THIRD_LANGUAGE_MALE INTEGER,THIRD_LANGUAGE_FEMALE INTEGER,
FOREIGN KEY (STATE_CODE) REFERENCES AREA(STATE_CODE),
PRIMARY KEY (STATE_CODE, GEOGRAPHIC_AREA, EDUCATION_LEVEL)
);'''
create_table(c,table_6,'LANG_EDUCATION')

done


In [55]:
data_3 = pd.read_csv('./multilingual-education.csv')
data_3.fillna(0)

,"C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State code,District code,Area Name,Total/Rural/Urban,Educational level,Number speaking second language,0,0,Number speaking third language,0,0
1,0,0,0,0,0,Persons,Males,Females,Persons,Males,Females
2,1,2,3,4,5,6,7,8,9,10,11
3,0,0,0,0,0,0,0,0,0,0,0
4,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
5,00,00,INDIA,Total,Illiterate,42266268,17851584,24414684,3879858,1890285,1989573
6,00,00,INDIA,Total,Literate,272722502,158844799,113877703,82129722,48646547,33483175
7,00,00,INDIA,Total,Literate but below primary,29345104,16126959,13218145,3733616,2108024,1625592
8,00,00,INDIA,Total,Primary but below middle,48570544,26588496,21982048,8636296,4782211,3854085
9,00,00,INDIA,Total,Middle but below matric/secondary,43116661,25214857,17901804,10743350,6266289,4477061


In [56]:
ids = data_3.iloc[4:,[0,3,4,5,6,7,8,9,10]]

In [57]:
schema_5 =['''''','''(?,?,?,?,?,?,?,?,?)''']
insert_many(c,schema_5,ids.values,'''LANG_EDUCATION''')

done


In [58]:
ids

,"C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
4,00,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
5,00,Total,Illiterate,42266268,17851584,24414684,3879858,1890285,1989573
6,00,Total,Literate,272722502,158844799,113877703,82129722,48646547,33483175
7,00,Total,Literate but below primary,29345104,16126959,13218145,3733616,2108024,1625592
8,00,Total,Primary but below middle,48570544,26588496,21982048,8636296,4782211,3854085
9,00,Total,Middle but below matric/secondary,43116661,25214857,17901804,10743350,6266289,4477061
10,00,Total,Matric/Secondary but below graduate,93204075,56022449,37181626,33899219,20411147,13488072
11,00,Total,Graduate and above,46958049,28285620,18672429,22146543,13316422,8830121
12,00,Rural,Total,162641485,92988052,69653433,35383989,21771062,13612927
13,00,Rural,Illiterate,30204008,12774715,17429293,2234076,1105619,1128457


## Queries

In [5]:
query_1 = ''' SELECT AREA.NAME,AREA.STATE_CODE 
FROM LANG_EDUCATION AS L , AGE_LITERACY  AS A, AREA 
WHERE L.STATE_CODE != '0' AND L.STATE_CODE = A.STATE_CODE AND AREA.STATE_CODE ==L.STATE_CODE AND
L.EDUCATION_LEVEL= 'Total' and A.AGE_GROUP ='All ages'  AND L.GEOGRAPHIC_AREA ='Total' AND A.GEOGRAPHIC_AREA ='Total' 
ORDER BY CAST(L.THIRD_LANGUAGE_TOTAL AS FLOAT)/CAST(A.TOTAL_PERSONS AS FLOAT) ASC'''

In [7]:
c.execute(query_1)
rows = c.fetchall()
for row in rows:
    print(row)

('State - UTTAR PRADESH', 9)
('State - RAJASTHAN', 8)
('State - BIHAR', 10)
('State - CHHATTISGARH', 22)
('State - MADHYA PRADESH', 23)
('State - UTTARAKHAND', 5)
('State - WEST BENGAL', 19)
('State - TAMIL NADU', 33)
('State - HARYANA', 6)
('State - MIZORAM', 15)
('State - JHARKHAND', 20)
('State - HIMACHAL PRADESH', 2)
('State - TRIPURA', 16)
('State - PUDUCHERRY', 34)
('State - ANDHRA PRADESH', 28)
('State - NCT OF DELHI', 7)
('State - MEGHALAYA', 17)
('State - KERALA', 32)
('State - KARNATAKA', 29)
('State - ODISHA', 21)
('State - ASSAM', 18)
('State - GUJARAT', 24)
('State - DAMAN & DIU', 25)
('State - LAKSHADWEEP', 31)
('State - JAMMU & KASHMIR', 1)
('State - MAHARASHTRA', 27)
('State - ANDAMAN & NICOBAR ISLANDS', 35)
('State - DADRA & NAGAR HAVELI', 26)
('State - MANIPUR', 14)
('State - NAGALAND', 13)
('State - PUNJAB', 3)
('State - SIKKIM', 11)
('State - ARUNACHAL PRADESH', 12)
('State - CHANDIGARH', 4)
('State - GOA', 30)


In [107]:
query_2 = ''' SELECT AGE_GROUP,MAX(S)  FROM (SELECT AGE_GROUP,SUM(SECOND_LANGUAGE_TOTAL+THIRD_LANGUAGE_TOTAL) AS S FROM LANGUAGE_AGE 
WHERE AGE_GROUP != 'Total' AND GEOGRAPHIC_AREA='Total' AND STATE_CODE !='0'
GROUP BY AGE_GROUP) 
'''

In [108]:
c.execute(query_2)
rows = c.fetchall()
for row in rows:
    print(row)

('30-49', 123910754)


In [8]:
query_3 = ''' SELECT AGE_GROUP, MAX(R) 
FROM (SELECT AGE_GROUP, MAX(CAST(TOTAL_MALE AS FLOAT)/CAST(TOTAL_FEMALE AS FLOAT)) AS R   
FROM AGE_LITERACY WHERE STATE_CODE = 0 AND GEOGRAPHIC_AREA = 'Total' AND AGE_GROUP != 'All ages'
UNION 
SELECT AGE_GROUP, MAX(CAST(TOTAL_FEMALE AS FLOAT)/CAST(TOTAL_MALE AS FLOAT)) AS R     
FROM AGE_LITERACY WHERE STATE_CODE = 0 AND GEOGRAPHIC_AREA = 'Total' AND AGE_GROUP != 'All ages')
'''
c.execute(query_3)
rows = c.fetchall()
for row in rows:
    print(row)

('18', 1.161050268927912)


In [36]:
query_4 = ''' SELECT A.TOTAL_PERSONS - L.SECOND_LANGUAGE_TOTAL-L.THIRD_LANGUAGE_TOTAL
FROM AGE_LITERACY AS A,LANG_EDUCATION AS L WHERE L.STATE_CODE = A.STATE_CODE AND L.EDUCATION_LEVEL = 'Total'
AND A.STATE_CODE = '0' AND L.GEOGRAPHIC_AREA='Total' AND A.GEOGRAPHIC_AREA = 'Total' AND A.AGE_GROUP = 'All ages'
'''

In [37]:
c.execute(query_4)
rows = c.fetchall()
for row in rows:
    print(row)

(809856627,)


In [66]:
query_5 = '''
SELECT AREA.NAME,MAX(R)
FROM AREA,(
SELECT STATE_CODE, CAST(SUM(AVG_AGE*TOTAL_PERSONS) AS FLOAT)/CAST(SUM(TOTAL_PERSONS) AS FLOAT) AS R
FROM AGE_LITERACY WHERE STATE_CODE != '0' AND GEOGRAPHIC_AREA = 'Total' AND AGE_GROUP!= 'All ages' AND AGE_GROUP!= 'Age not stated'
GROUP BY STATE_CODE) AS P
WHERE AREA.STATE_CODE = P.STATE_CODE
'''

In [67]:
c.execute(query_5)
rows = c.fetchall()
for row in rows:
    print(row)

('State - KERALA', 33.185498586746355)
